In [41]:
import sys
import os
from pathlib import Path

# Imports for vertex pipeline
from google.cloud import aiplatform
import google_cloud_pipeline_components
from google_cloud_pipeline_components import aiplatform as gcc_aip
from google_cloud_pipeline_components.v1.custom_job import CustomTrainingJobOp
from kfp.v2 import compiler
from kfp.v2.dsl import (
    Artifact,
    component,
    pipeline,
    Input,
    Output,
    Model,
    Dataset,
    InputPath,
    OutputPath,
)
import kfp.components as comp
import kfp.dsl as dsl
import warnings
warnings.filterwarnings("ignore")

from datetime import datetime

sys.path.append(str(Path(".").absolute().parent))
sys.path.append(str(Path(".").absolute().parent) + "/utils")
sys.path.append(str(Path(".").absolute().parent.parent))
sys.path.append(str(Path(".").absolute().parent.parent.parent))

import pipeline_utils
import argparse

try:
    args = pipeline_utils.get_args()
except:
    parser = argparse.ArgumentParser()
    parser.add_argument("--MODE", required=True, type=str)
    parser.add_argument("--STAGE1_FLAG", required=True, type=str)
    parser.add_argument("--ENSEMBLE_FLAG", required=True, type=str)
    parser.add_argument("--RF_CLF_MODEL_PATH", required=True, type=str)
    parser.add_argument("--LOGISTIC_CLF_MODEL_PATH", required=True, type=str)
    parser.add_argument("--STAGE1_NN_MODEL_PATH", required=True, type=str)
    parser.add_argument("--GNB_MODEL_PATH", required=True, type=str)
    parser.add_argument("--STG1_FEATURE_SELECTOR_MODEL_PATH", required=True, type=str)
    parser.add_argument("--NOSALES_MODEL_PATH", required=True, type=str)
    sys.args = [
        "--MODE", "test",
        "--STAGE1_FLAG", "train",
        "--ENSEMBLE_FLAG", "train",
        "--RF_CLF_MODEL_PATH", "",
        "--LOGISTIC_CLF_MODEL_PATH", "",
        "--STAGE1_NN_MODEL_PATH", "",
        "--GNB_MODEL_PATH", "",
        "--STG1_FEATURE_SELECTOR_MODEL_PATH", "",
        "--NOSALES_MODEL_PATH", "",
    ]
    args = parser.parse_args(sys.args)

PARAMS = pipeline_utils.yaml_import('settings.yml')

# Env flag for indentifying what env is used. 
ENV = PARAMS['env_flag']

# GCP Project id, service account, region, and docker images. 
PROJECT_ID = PARAMS['envs'][ENV]['PROJECT_ID']
SERVICE_ACCOUNT = PARAMS['envs'][ENV]['SERVICE_ACCOUNT']
REGION = PARAMS['envs'][ENV]['REGION']
BASE_IMAGE = PARAMS['envs'][ENV]['BASE_IMAGE']
MLFLOW_IMAGE = PARAMS['envs'][ENV]['MLFLOW_IMAGE']

# Training Pipeline.
RUN_PIPELINE = PARAMS['envs'][ENV]['RUN_PIPELINE']
PIPELINE_ROOT = "gs://oyi-ds-vertex-test-input-bucket" #PARAMS['envs'][ENV]['PIPELINE_ROOT']
PIPELINE_NAME = "test" #PARAMS['envs'][ENV]['PIPELINE_NAME']
PIPELINE_JSON = PIPELINE_NAME + ".json" # PARAMS['envs'][ENV]['PIPELINE_JSON']
TMP_PIPELINE_JSON = os.path.join("/tmp", PIPELINE_JSON)
# LATEST_PIPELINE_PATH = PARAMS['envs'][ENV]['LATEST_NOSALES_MODEL_PATH']
# LATEST_PIPELINE_PATH_JSON = LATEST_PIPELINE_PATH + ".json"
# LATEST_NOSALES_MODEL_PATH = LATEST_PIPELINE_PATH + ".json"


TRAINING_TABLE_NAME = PARAMS['envs'][ENV]['TRAINING_TABLE_NAME']
TRAINING_DATA_BQ_QUERY = f'select * from {TRAINING_TABLE_NAME}'

MLFLOW_EXP_NAME = PARAMS['envs'][ENV]['MLFLOW_EXP_NAME']
MODEL_REGISTRY_NAME = PARAMS['envs'][ENV]['MODEL_REGISTRY_NAME']

TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")
 
# Matches on non-word, non-regular-punctuation characters.
MATCHER = r"""[^a-zA-Z0-9'"!@#$%\^&*()\[\]{}:;<>?,.-=_+ ]+""" 

CLUB_THRESH_PATH = f"{PIPELINE_ROOT}/test" # PARAMS['envs'][ENV]['CLUB_THRESH_PATH']

print(f"ENV: {ENV} \nPROJECT_ID: {PROJECT_ID} \nBASE_IMAGE: {BASE_IMAGE} \nMLFLOW_IMAGE: {MLFLOW_IMAGE} \nPIPELINE_NAME: {PIPELINE_NAME} \nPIPELINE_JSON: {PIPELINE_JSON}")
print(f"\nTMP_PIPELINE_JSON: {TMP_PIPELINE_JSON} ") #\nLATEST_PIPELINE_PATH: {LATEST_PIPELINE_PATH} \nLATEST_PIPELINE_PATH_JSON: {LATEST_PIPELINE_PATH_JSON}")
print(f"\n\nPIPELINE_ROOT: {PIPELINE_ROOT} \nCLUB_THRESH_PATH: {CLUB_THRESH_PATH}")

ENV: dev 
PROJECT_ID: wmt-mlp-p-oyi-ds-or-oyi-dsns 
BASE_IMAGE: gcr.io/wmt-mlp-p-oyi-ds-or-oyi-dsns/oyi-vertex-pipeline-dev:latest 
MLFLOW_IMAGE: gcr.io/wmt-mlp-p-oyi-ds-or-oyi-dsns/mlflow-image-dev:latest 
PIPELINE_NAME: test 
PIPELINE_JSON: test.json

TMP_PIPELINE_JSON: /tmp/test.json 


PIPELINE_ROOT: gs://oyi-ds-vertex-test-input-bucket 
CLUB_THRESH_PATH: gs://oyi-ds-vertex-test-input-bucket/test


usage: ipykernel_launcher.py [-h] --MODE MODE --STAGE1_FLAG STAGE1_FLAG
                             --ENSEMBLE_FLAG ENSEMBLE_FLAG --RF_CLF_MODEL_PATH
                             RF_CLF_MODEL_PATH --LOGISTIC_CLF_MODEL_PATH
                             LOGISTIC_CLF_MODEL_PATH --STAGE1_NN_MODEL_PATH
                             STAGE1_NN_MODEL_PATH --GNB_MODEL_PATH
                             GNB_MODEL_PATH --STG1_FEATURE_SELECTOR_MODEL_PATH
                             STG1_FEATURE_SELECTOR_MODEL_PATH
                             --NOSALES_MODEL_PATH NOSALES_MODEL_PATH
ipykernel_launcher.py: error: the following arguments are required: --MODE, --STAGE1_FLAG, --ENSEMBLE_FLAG, --RF_CLF_MODEL_PATH, --LOGISTIC_CLF_MODEL_PATH, --STAGE1_NN_MODEL_PATH, --GNB_MODEL_PATH, --STG1_FEATURE_SELECTOR_MODEL_PATH, --NOSALES_MODEL_PATH


In [36]:
TRAINING_TABLE_NAME

'oyi_prod.oyi_train_no_testscan'

In [19]:
os.path.join(PIPELINE_ROOT, local_path)

'gs://oyi-ds-vertex-test-input-bucket/test_tpr.joblib'

In [80]:
import pandas as pd
from joblib import dump, load
import os
import pickle
from google.cloud import storage

data = {
  "calories": [420, 380, 390],
  "duration": [50, 40, 45]
}

df = pd.DataFrame.from_dict(data)
# local_path
# to_dump = joined.loc[joined['run_date']==joined.run_date.max()]
# to_dump = to_dump[select_col]
path = ''
level_name = "test"
local_path = level_name + '_tpr.joblib'
local_path_csv = level_name + '_tpr.csv'
csv_file_path = f"{PIPELINE_ROOT}/test_tpr_{ENV}"
dump(df, local_path)
storage_path = os.path.join(PIPELINE_ROOT, local_path)
print(storage_path)
blob = storage.blob.Blob.from_string(storage_path, client=storage.Client())

gs://oyi-ds-vertex-test-input-bucket/test_tpr.joblib


In [81]:
local_path, f"{csv_file_path}/{local_path_csv}"

('test_tpr.joblib',
 'gs://oyi-ds-vertex-test-input-bucket/test_tpr_dev/test_tpr.csv')

In [83]:
blob.upload_from_filename(local_path)
df.to_csv(f"{csv_file_path}/{local_path_csv}", index=False)

In [79]:
data.to_dataframe()

AttributeError: 'dict' object has no attribute 'to_dataframe'

In [21]:

@component(base_image=BASE_IMAGE)
def test1(out: Output[Dataset]):
    import pandas as pd
    
    data = {
      "calories": [420, 380, 390],
      "duration": [50, 40, 45]
    }
    df = pd.DataFrame(data)
    df.to_csv(out.path)
@component(base_image=BASE_IMAGE)
def test_save(pipeline_root: str, env: str):
    import pandas as pd
    from joblib import dump, load
    import os
    import pickle
    from google.cloud import storage
    
    data = {
      "calories": [420, 380, 390],
      "duration": [50, 40, 45]
    }
    # local_path
    # to_dump = joined.loc[joined['run_date']==joined.run_date.max()]
    # to_dump = to_dump[select_col]
    path = f"{pipeline_root}/test_all_level_tpr_{env}"
    level_name = "test"
    local_path = level_name + '_tpr.joblib'
    local_path_csv = level_name + '_tpr.csv'
    csv_file_path = f"{pipeline_root}/test_tpr_{env}"
    dump(data, local_path)
    storage_path = os.path.join(path, local_path)
    blob = storage.blob.Blob.from_string(storage_path, client=storage.Client())
    blob.upload_from_filename(local_path)
    
    data.to_csv(f"{csv_file_path}/{local_path_csv}", index=False)
        
@component(base_image=BASE_IMAGE, packages_to_install=["google_cloud_storage"])
def data_preprocessing(
        training_data_bq_query_input: str,
        matcher: str,
        project_id: str,
        env: str,
        pipeline_root: str,
        training_data_output: Output[Dataset]
    ):
    
    import pandas as pd
    from datetime import timedelta
    import utils
    from google.cloud import bigquery
    from google.cloud import storage

    client = bigquery.Client(project=project_id)
    data = client.query(training_data_bq_query_input).to_dataframe()
    nosales_data = data[
      (data.report_type!='C') &
      (data.display_ind == "Display") &
      (data.oh_qty>=0)]
    nosales_data["item_desc"] = nosales_data['item_desc'].str.replace(matcher, "", regex=True)
    nosales_data['run_date'] = pd.to_datetime(nosales_data['run_date'])
    max_date = nosales_data['run_date'].max()
    cutoff_date = (max_date - timedelta(days=182)).strftime('%Y-%m-%d')
    nosales_data = nosales_data[nosales_data.run_date > cutoff_date]
    
    nosales_data.replace("No Action Taken, already OFS", "No Action Taken, already out for sale", inplace=True)
    nosales_data.replace('Updated the NOSALES type with scrubber event', "No Action Taken, already out for sale", inplace=True)
    nosales_data.sort_values(by = ['run_date','club_nbr','item_nbr','event_ts'],inplace = True)
    nosales_data.drop_duplicates(['old_nbr','club_nbr','run_date'], keep='first',inplace = True)
    
    nosales_ext = utils.calculate_all_level_tpr(nosales_data, env, pipeline_root, save=False)
    nosales_ext.fillna(0, inplace=True)
    nosales_ext.to_csv(training_data_output.path, index=False)

@component(base_image=BASE_IMAGE)
def test(training_data_bq_query_input: str,
         matcher: str,
         project_id: str,
         env: str,
         pipeline_root: str,
         training_data_output: Output[Dataset]):
    import pandas as pd
    from datetime import timedelta
    import utils
    from google.cloud import bigquery
    from google.cloud import storage
    
    client = bigquery.Client(project=project_id)
    data = client.query(training_data_bq_query_input).to_dataframe()
    nosales_data = data[
      (data.report_type!='C') &
      (data.display_ind == "Display") &
      (data.oh_qty>=0)]
    nosales_data["item_desc"] = nosales_data['item_desc'].str.replace(matcher, "", regex=True)
    nosales_data['run_date'] = pd.to_datetime(nosales_data['run_date'])
    max_date = nosales_data['run_date'].max()
    cutoff_date = (max_date - timedelta(days=182)).strftime('%Y-%m-%d')
    nosales_data = nosales_data[nosales_data.run_date > cutoff_date]
    
    nosales_data.replace("No Action Taken, already OFS", "No Action Taken, already out for sale", inplace=True)
    nosales_data.replace('Updated the NOSALES type with scrubber event', "No Action Taken, already out for sale", inplace=True)
    nosales_data.sort_values(by = ['run_date','club_nbr','item_nbr','event_ts'],inplace = True)
    nosales_data.drop_duplicates(['old_nbr','club_nbr','run_date'], keep='first',inplace = True)
    
    nosales_ext = utils.calculate_all_level_tpr(nosales_data, env, pipeline_root, save=False)
    nosales_ext.fillna(0, inplace=True)
    nosales_ext.to_csv(training_data_output.path, index=False)
    
@component(base_image=BASE_IMAGE)
def test_upload(training_data_bq_query_input: str,
                matcher: str,
                project_id: str,
                env: str,
                pipeline_root: str,
                path_output: Output[Dataset]):
    import os
    import pandas as pd
    from datetime import timedelta
    import utils
    from google.cloud import bigquery
    from google.cloud import storage

    client = bigquery.Client(project=project_id)
    data = client.query(training_data_bq_query_input).to_dataframe()
    nosales_data = data[
      (data.report_type!='C') &
      (data.display_ind == "Display") &
      (data.oh_qty>=0)]
    nosales_data["item_desc"] = nosales_data['item_desc'].str.replace(matcher, "", regex=True)
    nosales_data['run_date'] = pd.to_datetime(nosales_data['run_date'])
    max_date = nosales_data['run_date'].max()
    cutoff_date = (max_date - timedelta(days=182)).strftime('%Y-%m-%d')
    nosales_data = nosales_data[nosales_data.run_date > cutoff_date]
    
    nosales_data.replace("No Action Taken, already OFS", "No Action Taken, already out for sale", inplace=True)
    nosales_data.replace('Updated the NOSALES type with scrubber event', "No Action Taken, already out for sale", inplace=True)
    nosales_data.sort_values(by = ['run_date','club_nbr','item_nbr','event_ts'],inplace = True)
    nosales_data.drop_duplicates(['old_nbr','club_nbr','run_date'], keep='first',inplace = True)
    
    nosales_ext = utils.calculate_all_level_tpr(nosales_data, env, pipeline_root, save=False)
    nosales_ext.fillna(0, inplace=True)
    nosales_ext.to_csv(path_output.path, index=False)
    
    
    # path_input_csv = os.path.join(path_input, "test.csv")
    # df_test = pd.read_csv(path_input_csv).drop(columns = 'nosales_club_thresh')
    # # club_threshold_output.path = path_input_csv
    # df_test.to_csv(club_threshold_output.path, index=False)
    

# @component(base_image=IMAGE_URI)
# def update_thresholds(
#     nosales_test_ext_input: Input[Dataset],
#     models_dir_path_input: str,
#     nosales_model_input: Input[Model],
#     club_threshold_outout: Output[Artifact]
# ):
    
#     import utils
#     import pandas as pd
#     from joblib import load, dump
#     import os
#     from google.cloud import storage
#     from tempfile import TemporaryFile
    
#     nosales_test_ext = pd.read_csv(nosales_test_ext_input.path)
#     nosales_test_ext['run_date'] = pd.to_datetime(nosales_test_ext['run_date'])
#     blob = storage.blob.Blob.from_string(nosales_model_input.path, client=storage.Client())
#     with TemporaryFile() as temp_file:
#         #download blob into temp file
#         blob.download_to_file(temp_file)
#         temp_file.seek(0)
#         #load into joblib
#         stack_pipeline=load(temp_file)
    
#     thresh = utils.gen_thresholds(df = nosales_test_ext,  predictions = stack_pipeline.predict_proba(X=nosales_test_ext), classes = stack_pipeline.classes_)
 
#     club_threshold_file_path = os.path.join(models_dir_path_input, "club_thresh_chain.joblib")
#     blob = storage.blob.Blob.from_string(club_threshold_file_path, client=storage.Client())
#     with TemporaryFile() as temp_file:
#         #download blob into temp file
#         blob.download_to_file(temp_file)
#         temp_file.seek(0)
#         #load into joblib
#         all_thresh=load(temp_file)
    
#     all_thresh['nosales_club_thresh'] = thresh
#     dump(all_thresh, "club_thresh_chain.joblib")
    
#     blob = storage.blob.Blob.from_string(club_threshold_output.path, client=storage.Client())
#     blob.upload_from_filename("club_thresh_chain.joblib")

In [ ]:

@component(base_image=BASE_IMAGE)
def update_thresholds(
        nosales_test_ext_input: Input[Dataset],
        club_thresh_path_input: str,
        nosales_model_input: Input[Model],
        club_threshold_output: Output[Dataset]
    ):
    
    import utils
    import pandas as pd
    import pickle
    import os
    from google.cloud import storage
    from tempfile import TemporaryFile
    
    nosales_test_ext = pd.read_csv(nosales_test_ext_input.path)
    nosales_test_ext['run_date'] = pd.to_datetime(nosales_test_ext['run_date'])
   
    with open(nosales_model_input.path, "rb") as handler:
        stack_pipeline = pickle.load(handler)
    
    nosales_thresh = utils.gen_thresholds(df = nosales_test_ext,  predictions = stack_pipeline.predict_proba(X=nosales_test_ext), classes = stack_pipeline.classes_)
    df_nosales_thresh = pd.DataFrame(nosales_thresh.items(), columns = ['club_nbr','nosales_club_thresh']) # nosales_club_thresh
    
    # club_threshold_file_path = os.path.join(club_thresh_path_input, "club_thresh_chain.csv")
    # df_cancelled_thresh = pd.read_csv(club_threshold_file_path).drop(columns = 'nosales_club_thresh')
    # all_thresh = df_cancelled_thresh.merge(df_nosales_thresh, how = 'left', on = 'club_nbr')
    # club_threshold_output.path = club_threshold_file_path
    # all_thresh.to_csv(club_threshold_file_path, index = False)
    
    club_threshold_file_path = os.path.join(club_thresh_path_input, "club_thresh_chain.csv")
    df_cancelled_thresh = pd.read_csv(club_threshold_file_path).drop(columns = 'nosales_club_thresh')
    all_thresh = df_cancelled_thresh.merge(df_nosales_thresh, how = 'left', on = 'club_nbr')
    all_thresh.to_csv(club_threshold_output.path, index = False)
    # df_output.to_csv(arg_path.path, index=False)

data = data_preprocessing(training_data_bq_query_input = TRAINING_DATA_BQ_QUERY,
                              matcher=MATCHER,
                              project_id = PROJECT_ID, 
                              env=ENV, 
                              pipeline_root=PIPELINE_ROOT)
    
train_test_data = train_test_split(nosales_ext_input=data.outputs['training_data_output'])

train_eval_data = train_eval_model(nosales_ext_input=data.outputs['training_data_output'],
                                   nosales_train_ext_input=train_test_data.outputs['nosales_train_ext_output'],
                                   nosales_test_ext_input=train_test_data.outputs['nosales_test_ext_output'],
                                   nosales_train_usampled_input=train_test_data.outputs['nosales_train_usampled_output'],
                                   mode=args.MODE,
                                   stage1_flag=args.STAGE1_FLAG,
                                   ensemble_flag=args.ENSEMBLE_FLAG,
                                   rf_clf_model_path_input=args.RF_CLF_MODEL_PATH,
                                   logistic_clf_model_path_input=args.LOGISTIC_CLF_MODEL_PATH,
                                   stage1_nn_model_path_input=args.STAGE1_NN_MODEL_PATH,
                                   gnb_model_path_input=args.GNB_MODEL_PATH,
                                   stg1_feature_selector_model_path_input=args.STG1_FEATURE_SELECTOR_MODEL_PATH,
                                   nosales_model_path_input=args.NOSALES_MODEL_PATH,
                                   latest_pipeline_path_input=LATEST_PIPELINE_PATH,
                                   project_id=PROJECT_ID,
                                   region=REGION,
                                   timestamp=TIMESTAMP)

updated_thresholds = update_thresholds(nosales_test_ext_input=train_eval_data.outputs['nosales_test_ext_output'],  
                                       club_thresh_path_input=CLUB_THRESH_PATH,
                                       nosales_model_input=train_eval_data.outputs['nosales_model_output'])

    

In [ ]:
df_no_sale = pd.read_csv(os.path.join(PARAMS['envs'][ENV]['CLUB_THRESH_PATH'], "club_thresh_chain.csv"))


In [ ]:
nosales_test_ext_path = "gs://oyi-ds-vertex-pipeline-bucket-nonprod/335163835346/oyi-nosales-model-pipeline-dev-20221028164657/train-eval-model_7275718579889111040/nosales_test_ext_output"
nosales_test_ext = pd.read_csv(nosales_test_ext_path)
nosales_test_ext['run_date'] = pd.to_datetime(nosales_test_ext['run_date'])




In [ ]:
# model_path = "gs://oyi-ds-vertex-pipeline-bucket-nonprod/335163835346/oyi-nosales-model-pipeline-dev-20221028164657/train-eval-model_7275718579889111040/nosales_model_output"
model_path = "gs://oyi-ds-vertex-pipeline-bucket-nonprod/335163835346/oyi-nosales-model-pipeline-dev-20221028164657/train-eval-model_7275718579889111040/nosales_model_output"
if not os.path.exists(model_path):
    raise FileNotFoundError("feature file not found: {0}".format(model_path))

with open(model_path, 'rb') as file:
    features = pickle.load(file)
    # return features
# with open(model_path, "rb") as handler:
#     stack_pipeline = pickle.load(handler)

In [ ]:
# nosales_thresh = utils.gen_thresholds(df = nosales_test_ext,  predictions = stack_pipeline.predict_proba(X=nosales_test_ext), classes = stack_pipeline.classes_)
#     df_nosales_thresh = pd.DataFrame(nosales_thresh.items(), columns = ['club_nbr','nosales_club_thresh']) # nosales_club_thresh

In [ ]:
nosales_test_ext['run_date'] = pd.to_datetime(nosales_test_ext['run_date'])
with open(nosales_model_input.path, "rb") as handler:
    stack_pipeline = pickle.load(handler)

In [ ]:
nosales_test_ext = pd.read_csv(nosales_test_ext_input.path)
nosales_test_ext['run_date'] = pd.to_datetime(nosales_test_ext['run_date'])

In [ ]:
def check():
    

In [ ]:
CLUB_THRESH_PATH

In [ ]:
PIPELINE_ROOT

In [ ]:
CLUB_THRESH_PATH,PARAMS['envs'][ENV]['CLUB_THRESH_PATH']

In [ ]:
import pandas as pd
# path = CLUB_THRESH_PATH
file_path = os.path.join(PARAMS['envs'][ENV]['CLUB_THRESH_PATH'], "club_thresh_chain.csv")
df_no_sale = pd.read_csv(file_path)
print(df_no_sale.count())

In [ ]:
df_no_sale

In [ ]:
CLUB_THRESH_PATH
# oyi-ds-vertex-test-input-bucket


In [ ]:

# club_threshold_file_path = os.path.join(club_thresh_path_input, "club_thresh_chain.csv")
# df_cancelled_thresh = pd.read_csv(club_threshold_file_path).drop(columns = 'nosales_club_thresh')
# all_thresh = df_cancelled_thresh.merge(df_nosales_thresh, how = 'left', on = 'club_nbr')
# club_threshold_output.path = club_threshold_file_path
# all_thresh.to_csv(club_threshold_file_path, index = False)

club_threshold_file_path = os.path.join(club_thresh_path_input, "club_thresh_chain.csv")
# df_cancelled_thresh = pd.read_csv(club_threshold_file_path).drop(columns = 'nosales_club_thresh')
all_thresh = df_cancelled_thresh.merge(df_nosales_thresh, how = 'left', on = 'club_nbr')
all_thresh.to_csv(club_threshold_output.path, index = False)

In [ ]:
# path_input_csv = os.path.join(CLUB_THRESH_PATH, "club_thresh_chain.csv")
# # print(path_input_csv)
# df_test = pd.read_csv(path_input_csv)

In [ ]:
# def check(training_data_bq_query_input: str,
#           matcher: str,
#           project_id: str,
#           env: str,
#           pipeline_root: str,
#           path_output: Output[Dataset]):
#     import os
#     import pandas as pd
    
#     path_input_csv = os.path.join(path_input, "test.csv")
#     df_test = pd.read_csv(path_input_csv)#.drop(columns = 'nosales_club_thresh')
#     print(df_test)
# check(path_input=CLUB_THRESH_PATH)

In [ ]:
CLUB_THRESH_PATH, PIPELINE_ROOT, PIPELINE_NAME

In [ ]:

# @dsl.pipeline(pipeline_root=PIPELINE_ROOT, name=PIPELINE_NAME)
# def pipeline():
#     data = data_preprocessing(training_data_bq_query_input = TRAINING_DATA_BQ_QUERY,
#                               matcher=MATCHER,
#                               project_id = PROJECT_ID, 
#                               env=ENV, 
#                               pipeline_root=PIPELINE_ROOT)

In [7]:
PIPELINE_ROOT

'gs://oyi-ds-vertex-test-input-bucket'

In [22]:
@dsl.pipeline(pipeline_root=PIPELINE_ROOT, name=PIPELINE_NAME)
def pipeline():
    # test1_job = test1()
    test_save_job = test_save(pipeline_root=PIPELINE_ROOT, env=ENV)

In [23]:
# @dsl.pipeline(pipeline_root=PIPELINE_ROOT, name=PIPELINE_NAME)
# def pipeline():
#     test1_job = test1()
    # test_job = test(training_data_bq_query_input=TRAINING_DATA_BQ_QUERY,
    #                           matcher=MATCHER,
    #                           project_id=PROJECT_ID, 
    #                           env=ENV, 
    #                           pipeline_root=PIPELINE_ROOT)
    
    # test_upload_job = test_upload(path_input=CLUB_THRESH_PATH)
    
#     train_test_data = train_test_split(nosales_ext_input=data.outputs['training_data_output'])
    
#     train_eval_data = train_eval_model(nosales_ext_input=data.outputs['training_data_output'],
#                                        nosales_train_ext_input=train_test_data.outputs['nosales_train_ext_output'],
#                                        nosales_test_ext_input=train_test_data.outputs['nosales_test_ext_output'],
#                                        nosales_train_usampled_input=train_test_data.outputs['nosales_train_usampled_output'],
#                                        mode=args.MODE,
#                                        stage1_flag=args.STAGE1_FLAG,
#                                        ensemble_flag=args.ENSEMBLE_FLAG,
#                                        rf_clf_model_path_input=args.RF_CLF_MODEL_PATH,
#                                        logistic_clf_model_path_input=args.LOGISTIC_CLF_MODEL_PATH,
#                                        stage1_nn_model_path_input=args.STAGE1_NN_MODEL_PATH,
#                                        gnb_model_path_input=args.GNB_MODEL_PATH,
#                                        stg1_feature_selector_model_path_input=args.STG1_FEATURE_SELECTOR_MODEL_PATH,
#                                        nosales_model_path_input=args.NOSALES_MODEL_PATH,
#                                        latest_pipeline_path_input=LATEST_PIPELINE_PATH,
#                                        project_id=PROJECT_ID,
#                                        region=REGION,
#                                        timestamp=TIMESTAMP)
   
    
    


In [24]:
TMP_PIPELINE_JSON

'/tmp/test.json'

In [25]:
compiler.Compiler().compile(pipeline_func=pipeline, package_path=TMP_PIPELINE_JSON)

In [26]:

pipeline_job = aiplatform.PipelineJob(
    display_name=f"{PIPELINE_NAME}-{TIMESTAMP}",
    template_path=TMP_PIPELINE_JSON,
    pipeline_root=PIPELINE_ROOT,
    parameter_values={},
    # labels={"user":"Jae"},
    enable_caching=False,
)

# PipelineJob(
#     display_name: str,
#     template_path: str,
#     job_id: Optional[str] = None,
#     pipeline_root: Optional[str] = None,
#     parameter_values: Optional[Dict[str, Any]] = None,
#     input_artifacts: Optional[Dict[str, str]] = None,
#     enable_caching: Optional[bool] = None,
#     encryption_spec_key_name: Optional[str] = None,
#     labels: Optional[Dict[str, str]] = None,
#     credentials: Optional[google.auth.credentials.Credentials] = None,
#     project: Optional[str] = None,
#     location: Optional[str] = None,
#     failure_policy: Optional[str] = None,
# )

In [27]:
pipeline_job.submit(service_account=SERVICE_ACCOUNT,network='projects/12856960411/global/networks/vpcnet-private-svc-access-usc1')

Creating PipelineJob
PipelineJob created. Resource name: projects/335163835346/locations/us-central1/pipelineJobs/test-20221029182856
To use this PipelineJob in another session:
pipeline_job = aiplatform.PipelineJob.get('projects/335163835346/locations/us-central1/pipelineJobs/test-20221029182856')
View Pipeline Job:
https://console.cloud.google.com/vertex-ai/locations/us-central1/pipelines/runs/test-20221029182856?project=335163835346


In [ ]:

##########################################

# pipeline_job = aiplatform.PipelineJob(
#     display_name=f"{PIPELINE_NAME}-{TIMESTAMP}",
#     template_path=TMP_PIPELINE_JSON,
#     pipeline_root=PIPELINE_ROOT,
#     parameter_values={},
#     enable_caching=False,
# )

# # pipeline_utils.store_pipeline(
# #     storage_path=LATEST_PIPELINE_PATH_JSON, 
# #     filename=TMP_PIPELINE_JSON
# # )

# pipeline_job.submit(service_account=SERVICE_ACCOUNT,network='projects/12856960411/global/networks/vpcnet-private-svc-access-usc1')

Downloading: 100%|██████████| 7/7 [00:01<00:00,  3.98rows/s]


,f0_
0,Abbey Grove
1,Abbey Grove
2,Abbey Road View
3,Abbey Road
4,Abbey Terrace
5,Abbey Wood Road
6,Abbey Grove


In [ ]:
SELECT regexp_replace(t, '.*[0-9]+[a-zA-Z]?[^a-zA-Z]*', '') FROM UNNEST(['23a, Abbey Grove','43a Abbey Grove','Block 509a Abbey Road View','511 Abbey Road','Flat 8a, Abbey Terrace','14 Abbey Wood Road','100 Abbey Grove']) t


In [37]:
TRAINING_DATA_BQ_QUERY = "SELECT * FROM `wmt-mlp-p-oyi-ds-or-oyi-dsns.oyi_prod.oyi_train_no_testscan`"

In [38]:
# SELECT * FROM `wmt-mlp-p-oyi-ds-or-oyi-dsns.oyi_prod.oyi_train_no_testscan` LIMIT 100


SyntaxError: invalid syntax (2501710994.py, line 1)

In [39]:
import pandas as pd
from datetime import timedelta
import utils
from google.cloud import bigquery
client = bigquery.Client(project=PROJECT_ID)

# data_preprocessing(training_data_bq_query_input = TRAINING_DATA_BQ_QUERY,
#                               matcher=MATCHER,
#                               project_id = PROJECT_ID, env=ENV, pipeline_root=PIPELINE_ROOT)

data = client.query(TRAINING_DATA_BQ_QUERY).to_dataframe()
# nosales_data = data[
#   (data.report_type!='C') &
#   (data.display_ind == "Display") &
#   (data.oh_qty>=0)]
# nosales_data["item_desc"] = nosales_data['item_desc'].str.replace(MATCHER, "", regex=True)
# nosales_data['run_date'] = pd.to_datetime(nosales_data['run_date'])
# max_date = nosales_data['run_date'].max()
# cutoff_date = (max_date - timedelta(days=182)).strftime('%Y-%m-%d')
# nosales_data = nosales_data[nosales_data.run_date > cutoff_date]

ArrowException: Unknown error: Wrapping SWEET�RED�ONION�6LB,6LB BIN failed

In [42]:
import pandas as pd
from datetime import timedelta
import utils
from google.cloud import bigquery
client = bigquery.Client(project=PROJECT_ID)
client.query(TRAINING_DATA_BQ_QUERY)

QueryJob<project=wmt-mlp-p-oyi-ds-or-oyi-dsns, location=US, id=a9cf4b0e-7a58-4f29-85d1-3c04fe1b5acd>

In [68]:
%%bigquery
WITH w AS (
      SELECT value AS word
      FROM STRING_SPLIT(N'RED,ONION', ',')
     )
SELECT t.*
FROM `wmt-mlp-p-oyi-ds-or-oyi-dsns.oyi_prod.oyi_train_no_testscan` t
WHERE NOT EXISTS (SELECT 1
                  FROM w
                  WHERE t.item_desc LIKE CONCAT('%', w.word, '%')
                 );

Executing query with job ID: f37d27e7-6c85-4e16-ac57-30e7a0b26bfd
Query executing: 0.22s


ERROR:
 400 Syntax error: Expected ")" or "," but got string literal 'RED,ONION' at [3:26]

Location: US
Job ID: f37d27e7-6c85-4e16-ac57-30e7a0b26bfd



In [61]:
%%bigquery
SELECT * FROM `wmt-mlp-p-oyi-ds-or-oyi-dsns.oyi_prod.oyi_train_no_testscan` WHERE item_desc LIKE '%ONION%'


Downloading: 100%|██████████| 4668/4668 [00:02<00:00, 2319.81rows/s]


ArrowException: Unknown error: Wrapping SWEET�RED�ONION�6LB,6LB BIN failed

In [74]:
%%bigquery
SELECT * FROM `wmt-mlp-p-oyi-ds-or-oyi-dsns.oyi_prod.oyi_train_no_testscan` LIMIT 10000 
# 1689514 


Downloading: 100%|██████████| 10000/10000 [00:01<00:00, 5411.06rows/s]


,club_nbr,item_nbr,old_nbr,days_no_sale,cat,subcat,state,unit_retail,oh_qty,avg_sales_interval,...,event_txt,event_user,event_ts,event_note,exception_type,central_dt,central_ts,ts_diff,spurious,action
0,4839,54132736,980129998,2.0,96,5,OK,31.98,18,0.395716,...,root_cause,bll0022.s04839,2022-09-05 20:06:47+00:00,"No Action Taken, already OFS",NOSALES,2022-09-05,2022-09-05 14:06:47,36,0,0
1,4843,22743296,754872,3.0,49,20,TX,4.82,14,3.965859,...,root_cause,wcswart.s04843,2022-08-09 15:08:50+00:00,"No Action Taken, already OFS",NOSALES,2022-08-09,2022-08-09 09:08:50,20,0,0
2,6239,22743296,754872,4.0,49,20,MD,4.82,15,4.272727,...,root_cause,oteuter.s06239,2022-09-04 00:55:52+00:00,"No Action Taken, already OFS",NOSALES,2022-09-03,2022-09-03 18:55:52,118,0,0
3,4817,22743296,754872,2.0,49,20,AL,4.82,10,6.506977,...,root_cause,jac007c.s04817,2022-08-25 16:45:20+00:00,Add to picklist,NOSALES,2022-08-25,2022-08-25 10:45:20,12,1,1
4,8190,22743296,754872,1.0,49,20,TX,4.82,38,9.404598,...,root_cause,awbass.s08190,2022-05-01 13:52:36+00:00,New price print sign has been printed,NOSALES,2022-05-01,2022-05-01 07:52:36,23,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,8152,53646862,980044441,1.0,8,50,OH,17.98,6,0.480631,...,root_cause,asadkin.s08152,2022-06-09 11:00:16+00:00,"No Action Taken, already OFS",NOSALES,2022-06-09,2022-06-09 05:00:16,1366,0,0
9996,4969,53646862,980044441,2.0,8,50,AR,17.98,16,0.614921,...,root_cause,G0G00HV.s04969,2022-09-30 15:05:30+00:00,"No Action Taken, already OFS",NOSALES,2022-09-30,2022-09-30 09:05:30,26,0,0
9997,8202,53646862,980044441,1.0,8,50,GA,17.98,2,1.008269,...,root_cause,SYSTEM,2022-05-29 10:50:15+00:00,Updated the on hands quantity for the item,NOSALES,2022-05-29,2022-05-29 04:50:15,0,1,1
9998,6867,53646862,980044441,1.0,8,50,TX,17.98,2,1.147892,...,root_cause,SYSTEM,2022-09-28 10:51:37+00:00,Updated the on hands quantity for the item,NOSALES,2022-09-28,2022-09-28 04:51:37,0,1,1


In [52]:
%%bigquery
COUNT (*) FROM oyi_prod.oyi_train_no_testscan

Executing query with job ID: 4f589852-9fc7-4627-87d0-911b209352d7
Query executing: 0.24s


ERROR:
 400 Syntax error: Expected end of input but got identifier "COUNT" at [1:1]

Location: US
Job ID: 4f589852-9fc7-4627-87d0-911b209352d7



In [43]:
client.query(TRAINING_DATA_BQ_QUERY).to_dataframe()

ArrowException: Unknown error: Wrapping SWEET�RED�ONION�6LB,6LB BIN failed